# Global Impact of COVID-19

In [90]:
# importing libraries

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

> In order to start with the Exploratory part, we believe it would be a good idea to showcase some **global impacts of COVID-19**. 

> The corresponding source for the datasets used further below can be found right from the [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series) repositiory, which is supposed to be updated every single day and get us a run time evaluations and assess the plots with the most updated and legitimate data.


In [91]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

## Data preparation

Proper data preparation allows for efficient analysis - it helps eliminate errors and inaccuracies that could have occurred during the data gathering process and can therefore help in removing some bias resulting from poor data quality. Loading data, cleaning data (removing unnecessary data), transforming data formats, and re-arranging data are the various steps involved in the data preparation step.

> The corresponding available and raw data had the initials of column in the upper case, the below section helps us rename the column names in lowercase. 

In [92]:
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
country_df.head()

,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
0,Afghanistan,2020-11-24 23:27:22,33.93911,67.709953,45280.0,1712.0,36122.0,7446.0,116.316285,NaN,NaN,3.780919,4,AFG
1,Albania,2020-11-24 23:27:22,41.15330,20.168300,34300.0,735.0,16666.0,16899.0,1191.882688,NaN,NaN,2.142857,8,ALB
2,Algeria,2020-11-24 23:27:22,28.03390,1.659600,77000.0,2309.0,50070.0,24621.0,175.594455,NaN,NaN,2.998701,12,DZA
3,Andorra,2020-11-24 23:27:22,42.50630,1.521800,6351.0,76.0,5503.0,772.0,8219.763153,NaN,NaN,1.196662,20,AND
4,Angola,2020-11-24 23:27:22,-11.20270,17.873900,14742.0,338.0,7444.0,6960.0,44.854499,NaN,NaN,2.292769,24,AGO


> Record columns from the files are supposed to be in a uniformity, the below section is used to convert the column name of few of the existing columns to another for further convenience.

In [93]:
# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
0,Afghanistan,2020-11-24 23:27:22,33.93911,67.709953,45280.0,1712.0,36122.0,7446.0,116.316285,NaN,NaN,3.780919,4,AFG
1,Albania,2020-11-24 23:27:22,41.15330,20.168300,34300.0,735.0,16666.0,16899.0,1191.882688,NaN,NaN,2.142857,8,ALB
2,Algeria,2020-11-24 23:27:22,28.03390,1.659600,77000.0,2309.0,50070.0,24621.0,175.594455,NaN,NaN,2.998701,12,DZA
3,Andorra,2020-11-24 23:27:22,42.50630,1.521800,6351.0,76.0,5503.0,772.0,8219.763153,NaN,NaN,1.196662,20,AND
4,Angola,2020-11-24 23:27:22,-11.20270,17.873900,14742.0,338.0,7444.0,6960.0,44.854499,NaN,NaN,2.292769,24,AGO


> It is supposed that the files are prepared to be browsed as displayed below in sequence for `confirmed`, `death` and `recovered` cases.

In [94]:
confirmed_df.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,40026,40073,40141,40200,40287,40357,40510,40626,40687,40768,40833,40937,41032,41145,41268,41334,41425,41501,41633,41728,41814,41935,41975,42033,42092,42297,42463,42609,42795,42969,43035,43240,43403,43628,43851,44228,44443,44503,44706,44988
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16212,16501,16774,17055,17350,17651,17948,18250,18556,18858,19157,19445,19729,20040,20315,20634,20875,21202,21523,21904,22300,22721,23210,23705,24206,24731,25294,25801,26211,26701,27233,27830,28432,29126,29837,30623,31459,32196,32761,33556
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,53777,53998,54203,54402,54616,54829,55081,55357,55630,55880,56143,56419,56706,57026,57332,57651,57942,58272,58574,58979,59527,60169,60800,61381,62051,62693,63446,64257,65108,65975,66819,67679,68589,69591,70629,71652,72755,73774,74862,75867
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3190,3377,3377,3377,3623,3623,3811,3811,4038,4038,4038,4325,4410,4517,4567,4665,4756,4825,4888,4910,5045,5135,5135,5319,5383,5437,5477,5567,5616,5725,5725,5872,5914,5951,6018,6066,6142,6207,6256,6304
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7096,7222,7462,7622,7829,8049,8338,8582,8829,9026,9381,9644,9871,10074,10269,10558,10805,11035,11228,11577,11813,12102,12223,12335,12433,12680,12816,12953,13053,13228,13374,13451,13615,13818,13922,14134,14267,14413,14493,14634


In [95]:
death_df.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1481,1485,1488,1492,1497,1499,1501,1505,1507,1511,1514,1518,1523,1529,1532,1533,1536,1536,1541,1544,1548,1554,1554,1556,1558,1574,1577,1581,1591,1595,1605,1617,1626,1638,1645,1650,1661,1675,1687,1695
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,439,443,448,451,454,458,462,465,469,473,477,480,487,493,499,502,509,518,527,532,536,543,549,557,559,571,579,590,598,605,612,623,631,637,646,657,672,685,699,716
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1827,1841,1846,1856,1865,1873,1880,1888,1897,1907,1914,1922,1931,1941,1949,1956,1964,1973,1980,1980,1999,2011,2024,2036,2048,2062,2077,2093,2111,2124,2139,2154,2168,2186,2206,2224,2236,2255,2272,2294
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,59,59,59,59,62,62,63,63,69,69,69,72,72,72,73,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,76,76,76,76,76,76,76,76,76
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,228,234,241,247,248,251,255,260,265,267,268,270,271,275,275,279,284,286,289,291,296,299,300,303,307,308,308,312,315,317,322,322,324,328,332,333,334,336,337,337


In [96]:
recovered_df.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,40026,40073,40141,40200,40287,40357,40510,40626,40687,40768,40833,40937,41032,41145,41268,41334,41425,41501,41633,41728,41814,41935,41975,42033,42092,42297,42463,42609,42795,42969,43035,43240,43403,43628,43851,44228,44443,44503,44706,44988
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16212,16501,16774,17055,17350,17651,17948,18250,18556,18858,19157,19445,19729,20040,20315,20634,20875,21202,21523,21904,22300,22721,23210,23705,24206,24731,25294,25801,26211,26701,27233,27830,28432,29126,29837,30623,31459,32196,32761,33556
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,53777,53998,54203,54402,54616,54829,55081,55357,55630,55880,56143,56419,56706,57026,57332,57651,57942,58272,58574,58979,59527,60169,60800,61381,62051,62693,63446,64257,65108,65975,66819,67679,68589,69591,70629,71652,72755,73774,74862,75867
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3190,3377,3377,3377,3623,3623,3811,3811,4038,4038,4038,4325,4410,4517,4567,4665,4756,4825,4888,4910,5045,5135,5135,5319,5383,5437,5477,5567,5616,5725,5725,5872,5914,5951,6018,6066,6142,6207,6256,6304
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7096,7222,7462,7622,7829,8049,8338,8582,8829,9026,9381,9644,9871,10074,10269,10558,10805,11035,11228,11577,11813,12102,12223,12335,12433,12680,12816,12953,13053,13228,13374,13451,13615,13818,13922,14134,14267,14413,14493,14634


## COVID-19 impact on countries

Let us know probe deeper into the COVID-19 impact on certain countries around the globe and get some useful insights.

> In order to start with, below section represents the `confirmed`, `death`, `recovered` and `active` cases for few countries.

> The respective classified sub-sections are further highlighted using the different set of colors in order to obtain certain uniformity throughout the particular sections of the exploratory part.

In [97]:
# sorting the values by confirmed descending order
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    c = 'background-color: cyan'  
    r = 'background-color: pink'
    g = 'background-color: lightgreen'
    y = 'background-color: yellow'
    o = 'background-color: orange'

    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = c
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    df1.iloc[:, 7] = y
    df1.iloc[:, 11] = o    
    return df1

def latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(latest_cases, n='5') #variable

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='5', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

> It can be noticed that the above table is interactive in accordance to the number of countries, willing to be showcased once changed the variable, maintaining the sequence of the countries with highest number of confirmed cases.

In [98]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

### Severe impact on the countries 

Below section covers the severe impact on the countries, arranged in a scatter plot, showcasing highly affected countries in the descending order.

In [99]:
#plotting the 20 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" impacted countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    height=600,
    width=900
    )
    fig.show();

interact(bubble_chart, n=20)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=20, description='n', max=60, min=-20), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

> It can be noticed that an interactive button is available to fix a scale for number of countries, anyone wants to display between certain range. Apparently, it can be seen that the section is currently able to accommodate details for 20 countries in the above plot, however, the range of the value `n` is variable and can be reduced to as small as `1` or can be maximimzed to value `50` for insightful visualization.

### Time series analysis for particular country

The respective section helps to visualize the certainity in growth of `confirmed cases` & `deaths` throughout the time right from when the pandemic started to evolve. 

In [100]:
def timeseries_per_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['cyan', 'pink']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID-19 cases of " + country,
        xaxis_title='Date (Weekly)',
        yaxis_title='Number of Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 900,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();


> The below section helps to interact with the function from the original piece of code to customize and add the country of the User's choice. In the case of default, the section displays the plot for whole `World` which can be further changed to the country of User's choice.

> Hovering the cursor over the respective graph helps user gets idea about the more detailed information, eg - count of `confirmed cases` or `deaths` for a particular day/week.

In [101]:
interact(timeseries_per_country, country='World')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' 
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

## 20 worst impacted countries

> The analogy in the respective section revolves around the different sub-sections like `confirmed cases`, `deaths`, `recovered cases`, `active cases` and `mortality rate` for first 20 impacted countries around the globe.

> It is advisable to look at each individual plot for classified sub-section of the corresponding section. A pattern could be acknowledged at the end of the section for further analysis. 


### Analysis on Confirmed cases

In [102]:
px.bar(
    sorted_country_df.head(20),
    x = "country",
    y = "confirmed",
    title= "First 20 affected countries with highest confirm cases", 
    color_discrete_sequence=["cyan"], 
    height=600,
    width=900
)

> From the above plot, it can be easily derived that `US`, `India` and `Brazil` are among the countries which are highly affected due to COVID-19.   

> The respective plot showcases the top 20 countries with large number of`confirmed cases` and implies that these are the countries which are supposed to work on the health regulations in order to bend the curve. 

### Analysis on Deaths

In [103]:
px.bar(
    sorted_country_df.head(20),
    x = "country",
    y = "deaths",
    title= "First 20 affected countries with highest confirm cases showcasing huge volume of deaths", 
    color_discrete_sequence=["pink"], 
    height=600,
    width=900
)

> From the above plot, it can be said that `US`, `Brazil`, `India` and `Mexico`are highly impacted countries when it comes to `deaths`.

### Analysis on recovered cases

In [104]:
px.bar(
    sorted_country_df.head(20),
    x = "country",
    y = "recovered",
    title= "First 20 countries with highest confirm cases showcasing effective recovery rate", 
    color_discrete_sequence=["lightgreen"], 
    height=600,
    width=900
)

> The respective graph showcases the recovery rate for particular countries, however, it can be concluded that `India` has the highest `recovered cases` when compared to any other country in the list of top 20 affected countries. 

> Following to `India`, `Brazil` also surprizingly has high volume of `recovered cases` later followed by `US` and then other remaining countries with less count and even lesser impact.

### Analysis on Active cases



In [105]:
px.bar(
    sorted_country_df.head(20),
    x = "country",
    y = "active",
    title= "Firt 20 impacted countries with highest confirm cases showcasing active patients",
    color_discrete_sequence=["yellow"], 
    height=600,
    width=900
)

> The corresponding plot refers to the `active cases` among the top 20 countries with highest confirmed cases.

> It can be summarised that, `US` tends to peak the result for highest `active cases` as of latest followed by `France`, `Spain`, `United Kingdom`, `Italy`, `Belgium` , `India` and `Brazi`. 

### Analysis on Mortality rate

In [106]:
px.bar(
    sorted_country_df.head(20),
    x = "country",
    y = "mortality_rate",
    title= "First 20 countries with highest confirm cases showcasing mortality rate",
    color_discrete_sequence=["orange"], 
    height=600,
    width=900
)

## Global spread of COVID-19

> The below map shows the geographic expressions for each country around the horizon with significant data for `Confirmed cases`, `confirmed deaths`, and tries to showcase `death rate` for the respective countries.

In [107]:

world_map = folium.Map(location=[10,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='black',
        fill_color='grey',
        tooltip = "<div style='margin: 0; background-color: grey; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+  
                    "</ul></div>",
        ).add_to(world_map)

world_map